In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np


In [3]:
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,HistGradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier,BaggingClassifier,StackingClassifier,IsolationForest,VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

In [4]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv('test.csv')

data = df_train

In [5]:
data.isna().sum().sort_values()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64

In [6]:
def title(word):
    parts = word.split(",")
    if len(parts )> 1 and "Mr" in parts[1]:
        return 0
    else:
        return 1
    
data['title']=data["Name"].apply(title)
    

In [7]:
data = data.drop(["PassengerId",'Name','Cabin'],axis=1)
data.shape

(891, 10)

In [8]:
data.isna().sum().sort_values()

Survived      0
Pclass        0
Sex           0
SibSp         0
Parch         0
Ticket        0
Fare          0
title         0
Embarked      2
Age         177
dtype: int64

In [9]:
mean_fare = np.mean(data['Fare'])
data['Fare']=data['Fare'].fillna(mean_fare)
data['Age']=data['Age'].fillna(np.mean(data['Age']))
data =data.dropna(subset=['Embarked'])

In [10]:
data
def age_adult_child(a):
    if a <= 15:
        return 1
    else:
        return 0
data['Age']=data['Age'].apply(age_adult_child)

In [11]:

data['Age']=data['Age'].astype('int')

In [12]:
data = data.drop(['Ticket'],axis=1)
def assigner(a):
    if a == True:
        return 1
    else:
        return 0
data = pd.get_dummies(data,columns=['Sex','Embarked'])
data



,Survived,Pclass,Age,SibSp,Parch,Fare,title,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,1,0,7.2500,0,False,True,False,False,True
1,1,1,0,1,0,71.2833,0,True,False,True,False,False
2,1,3,0,0,0,7.9250,1,True,False,False,False,True
3,1,1,0,1,0,53.1000,0,True,False,False,False,True
4,0,3,0,0,0,8.0500,0,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,0,13.0000,1,False,True,False,False,True
887,1,1,0,0,0,30.0000,1,True,False,False,False,True
888,0,3,0,1,2,23.4500,1,True,False,False,False,True
889,1,1,0,0,0,30.0000,0,False,True,True,False,False


In [13]:
for i in ['Sex_male','Sex_female','Embarked_C','Embarked_Q','Embarked_S']:
    data[i]=data[i].apply(assigner)
data

,Survived,Pclass,Age,SibSp,Parch,Fare,title,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,1,0,7.2500,0,0,1,0,0,1
1,1,1,0,1,0,71.2833,0,1,0,1,0,0
2,1,3,0,0,0,7.9250,1,1,0,0,0,1
3,1,1,0,1,0,53.1000,0,1,0,0,0,1
4,0,3,0,0,0,8.0500,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,0,13.0000,1,0,1,0,0,1
887,1,1,0,0,0,30.0000,1,1,0,0,0,1
888,0,3,0,1,2,23.4500,1,1,0,0,0,1
889,1,1,0,0,0,30.0000,0,0,1,1,0,0


In [14]:
#changing class weights
data['Pclass'] = data['Pclass'].map({1:3, 2:2, 3: 1})
data

,Survived,Pclass,Age,SibSp,Parch,Fare,title,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0,1,0,7.2500,0,0,1,0,0,1
1,1,3,0,1,0,71.2833,0,1,0,1,0,0
2,1,1,0,0,0,7.9250,1,1,0,0,0,1
3,1,3,0,1,0,53.1000,0,1,0,0,0,1
4,0,1,0,0,0,8.0500,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,0,13.0000,1,0,1,0,0,1
887,1,3,0,0,0,30.0000,1,1,0,0,0,1
888,0,1,0,1,2,23.4500,1,1,0,0,0,1
889,1,3,0,0,0,30.0000,0,0,1,1,0,0


In [15]:
X_train = data.drop(['Survived'],axis=1).values
Y_train = data['Survived'].values


In [27]:
model = {'LogisticRegression':0,'RandomForestClassifier':0,'Xgboost':0}
logreg = LogisticRegression(solver='liblinear')
Rclf = RandomForestClassifier()
Xgboost = GradientBoostingClassifier(subsample= 0.8, n_estimators= 100, min_samples_split= 10, min_samples_leaf= 1,max_features= None, max_depth= 5, learning_rate= 0.01)
his = HistGradientBoostingClassifier()
ex = ExtraTreesClassifier()
ada =AdaBoostClassifier()
Bas = BaggingClassifier()
#Vot = VotingClassifier()
models =[logreg,Rclf,Xgboost,his,ex,ada,Bas]
scores =list()
for i in models:
    Kf = KFold(n_splits=5,shuffle=True,random_state=42)
    cross = cross_val_score(i,X_train,Y_train,cv=Kf)
    scores.append(np.mean(cross))
print(scores)
    


[0.7963943375864915, 0.8053831016314353, 0.8301847267187202, 0.8166127086904081, 0.8031422586174062, 0.8053767536342283, 0.8200215831905033]


In [17]:
data = df_test
data['title']=data["Name"].apply(title)
data = data.drop(["PassengerId",'Name','Cabin'],axis=1)
mean_fare = np.mean(data['Fare'])
data['Fare']=data['Fare'].fillna(mean_fare)
data['Age']=data['Age'].fillna(np.mean(data['Age']))
data =data.dropna(subset=['Embarked'])
data['Age']=data['Age'].apply(age_adult_child)

In [18]:
data.isna().sum().sort_values()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
title       0
dtype: int64

In [19]:
data = pd.get_dummies(data,columns=['Sex','Embarked'])
for i in ['Sex_male','Sex_female','Embarked_C','Embarked_Q','Embarked_S']:
    data[i]=data[i].apply(assigner)
data['Pclass'] = data['Pclass'].map({1:3, 2:2, 3: 1})
data

,Pclass,Age,SibSp,Parch,Ticket,Fare,title,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,0,0,330911,7.8292,0,0,1,0,1,0
1,1,0,1,0,363272,7.0000,0,1,0,0,0,1
2,2,0,0,0,240276,9.6875,0,0,1,0,1,0
3,1,0,0,0,315154,8.6625,0,0,1,0,0,1
4,1,0,1,1,3101298,12.2875,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1,0,0,0,A.5. 3236,8.0500,0,0,1,0,0,1
414,3,0,0,0,PC 17758,108.9000,1,1,0,1,0,0
415,1,0,0,0,SOTON/O.Q. 3101262,7.2500,0,0,1,0,0,1
416,1,0,0,0,359309,8.0500,0,0,1,0,0,1


In [28]:
#grid search
data = data.drop(['Ticket'],axis=1)
data

,Pclass,Age,SibSp,Parch,Fare,title,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,0,0,7.8292,0,0,1,0,1,0
1,1,0,1,0,7.0000,0,1,0,0,0,1
2,2,0,0,0,9.6875,0,0,1,0,1,0
3,1,0,0,0,8.6625,0,0,1,0,0,1
4,1,0,1,1,12.2875,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1,0,0,0,8.0500,0,0,1,0,0,1
414,3,0,0,0,108.9000,1,1,0,1,0,0
415,1,0,0,0,7.2500,0,0,1,0,0,1
416,1,0,0,0,8.0500,0,0,1,0,0,1


In [26]:
from sklearn.model_selection import RandomizedSearchCV



# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize GridSearchCV
grid_search = RandomizedSearchCV(estimator=Xgboost, param_distributions=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, Y_train)

# Best parameters
best_params = grid_search.best_params_

# Best estimator
best_model = grid_search.best_estimator_

print("Best Parameters:", best_params)




Best Parameters: {'subsample': 0.8, 'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 5, 'learning_rate': 0.01}


In [33]:
Xgboost.fit(X_train,Y_train)
Xgboost.score(X_train,Y_train)
x = Xgboost.predict(data)

c:\Users\HP\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [34]:
submission=pd.DataFrame({"PassengerId":df_test['PassengerId'],"Survived": x})

In [35]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
